In [ ]:
import numpy as np
#moviepy==1.0.3 imageio==2.31.5 imageio-ffmpeg
# from moviepy import *

In [ ]:
from moviepy.video.VideoClip import VideoClip
from moviepy.video.compositing.CompositeVideoClip import CompositeVideoClip
from PIL import Image

def create_zoom_pan_video_manual(image_path, output_path, duration=180, fps=30):
    """
    Creates a video with a zoom-in and pan-from-right-to-left effect on a large image
    by manually generating each frame using Pillow. This version is compatible with the latest moviepy API.

    Args:
        image_path (str): Path to the large input image.
        output_path (str): Path to save the output video file (e.g., 'output.mp4').
        duration (int): The duration of the video in seconds (3 minutes = 180 seconds).
        fps (int): Frames per second of the output video.
    """
    
    # --- Video and Image Parameters ---
    video_width = 1920
    video_height = 1080
    initial_scale = 3.0
    final_scale = 1.0

    # 1. Load the original image once using Pillow
    original_pil_image = Image.open(image_path)
    image_width, image_height = original_pil_image.size

    # --- Frame Generation Function ---
    def make_frame(t):
        """
        This function is called by VideoClip to generate each frame at time t.
        It calculates the zoom and pan, resizes the image, and places it on a canvas.
        """
        # Calculate the scale factor at time t (from 3.0 to 1.0)
        scale = initial_scale + (final_scale - initial_scale) * (t / duration)
        
        # Calculate the new size of the frame
        new_width = int(image_width * scale)
        new_height = int(image_height * scale)
        
        # Resize the image using Pillow's high-quality resampling
        resized_pil_image = original_pil_image.resize((new_width, new_height), Image.Resampling.LANCZOS)
        
        # Calculate the position (x, y)
        x_start = video_width - image_width * initial_scale
        x_end = 0
        x = int(x_start + (x_end - x_start) * (t / duration))
        
        y = int((video_height - new_height) / 2)
        
        # Create a blank black canvas for the frame
        frame_canvas = Image.new('RGB', (video_width, video_height), (0, 0, 0))
        
        # Paste the resized image onto the canvas at the calculated position
        frame_canvas.paste(resized_pil_image, (x, y))
        
        # Return the frame as a NumPy array
        return np.array(frame_canvas)

    # --- Create the Video Clip ---
    # 2. Create a VideoClip with our make_frame function and set the duration
    animated_clip = VideoClip(make_frame, duration=duration)

    # --- Write the Video File ---
    # 3. Write the final video file
    print("Writing video file...")
    animated_clip.write_videofile(output_path, fps=fps, codec='libx264')
    print(f"Video '{output_path}' created successfully!")

# Example usage:
if __name__ == '__main__':
    image_file = 'big_img_4k_3840x2160.png'
    output_video = 'zoom_pan_video.mp4'

    import os
    if not os.path.exists(image_file):
        print(f"Error: The image file '{image_file}' was not found.")
        print("Please place a large image file in the same directory and name it 'large_image.jpg'.")
    else:
        create_zoom_pan_video(image_file, output_video)

AttributeError: 'ImageClip' object has no attribute 'resize'